# PreProcess

---



## Read DataSet

In [13]:
with open ("hafez.txt", "r") as dataset:
    data = dataset.read().splitlines()

In [ ]:
print("dataLen: ", len(data))
data

## Read Stop Words

In [15]:
with open ("fa_stop_words.txt", "r") as fa_stop_words:
    stop_words = fa_stop_words.read().splitlines()

In [ ]:
stop_words

## Normalize with Hazm

In [ ]:
pip install hazm

In [3]:
from hazm import *

In [8]:
normalizer = Normalizer()
normalized_data = []
for line in data:
  normalized_data.append(normalizer.normalize(line))

In [ ]:
normalized_data

## Tokenize with Hazm

In [10]:
tokenized_data = []
for line in normalized_data:
  tokenized_data.append(word_tokenize(line))

In [ ]:
tokenized_data

## Remove Stop Words

In [17]:
clear_data = []
for line in tokenized_data:
  clear_line = []
  for word in line:
    if word not in stop_words:
      clear_line.append(word)
  clear_data.append(clear_line)

In [ ]:
clear_data